<a href="https://colab.research.google.com/github/bbanzai88/Artificial_Influencers/blob/main/Copy_of_AI_Influencer_Ollama_Colab_VisualFixBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 AI Influencer with Ollama (Colab Version)
This notebook runs a local LLM (e.g., `deepseek-r1:1.5b`) using Ollama in Colab to simulate an influencer generating text content and imagery.

**Modules:**
- Ollama LLM for captioning and replies
- Stable Diffusion for image generation
- Automated comment reply generation

In [ ]:
# 🛠️ Step 1: Install & launch Ollama locally in Colab
!pip install --quiet langchain-ollama python-docx nest_asyncio tqdm

import os, threading, subprocess, time, requests

for v in ["OPENAI_API_KEY","LITELLM_PROVIDER","LITELL M_MODEL","LITELL M_BASE_URL"]:
    os.environ.pop(v, None)
os.environ["OLLAMA_HOST"] = "127.0.0.1:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

!curl -fsSL https://ollama.com/install.sh -o install.sh
!bash install.sh && chmod +x /usr/local/bin/ollama

def _serve_ollama():
    subprocess.Popen(["ollama", "serve"], stderr=subprocess.DEVNULL)
threading.Thread(target=_serve_ollama, daemon=True).start()
time.sleep(8)

print("✅ Ollama status:", requests.get("http://127.0.0.1:11434").status_code)

In [ ]:
# 📥 Step 2: Pull the deepseek-r1:1.5b model
!ollama pull deepseek-r1:1.5b

## 🧠 Step 3: Define the AI Influencer Persona

In [ ]:
persona = {
    'name': 'Ava Monroe',
    'style': 'vintage fashion',
    'tone': 'witty and confident'
}

prompt = (
    f"You are {persona['name']}, an AI fashion influencer known for {persona['style']} looks. "
    f"Write a witty Instagram caption about wearing a vintage leather jacket on a crisp autumn day."
)

## 💬 Step 4: Generate Caption Using Ollama

In [ ]:
import requests

response = requests.post(
    "http://127.0.0.1:11434/api/generate",
    json={
        "model": "deepseek-r1:1.5b",
        "prompt": prompt,
        "stream": False
    }
)

print("📝 Caption:", response.json()['response'])

## 🎨 Step 5: Generate Image Using Stable Diffusion

In [ ]:
!pip install --quiet diffusers transformers accelerate safetensors

from diffusers import StableDiffusionPipeline
import torch
from IPython.display import display

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

image_prompt = "Fashionable young woman wearing a vintage leather jacket, autumn city background, photorealistic"
image = pipe(image_prompt).images[0]
image.save("ava_monroe_image.png")
display(image)

## 🤖 Step 6: Generate Automated Comment Replies

In [ ]:
comments = [
    "Love your style! 😍",
    "Where did you get that jacket?",
    "You always know how to rock vintage looks!"
]

for comment in comments:
    reply_prompt = (
        f"As Ava Monroe, a witty and confident fashion influencer, reply briefly to this fan comment: "
        f"'{comment}'\n"
        f"Your response should be no more than 25 words. Do NOT include any <think> or internal reasoning."
    )

    response = requests.post(
        "http://127.0.0.1:11434/api/generate",
        json={
            "model": "deepseek-r1:1.5b",
            "prompt": reply_prompt,
            "stream": False
        }
    )

    print(f"🗨️ {comment}\n💬 {response.json()['response'].strip()}\n")